### Kalman filter

Heavily inspired by weekly project week 7 and exercise 2 week 7

In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

**Update and predict** linear algebra of kalman filters.

In [7]:
def update(x, P, Z, H, R):
    ### Insert update function
    y = Z - H@x
    s = H@P@H.T + R
    K = P@H.T@np.linalg.pinv(s)
    new_x = x + K@y
    new_P = (np.identity((K@H).shape[0]) - K@H)@P
    return new_x, new_P
    
def predict(x, P, F, u, Q):
    ### insert predict function
    new_x = F@x + u
    new_P = F@P@F.T + Q # Q=0. No internal disturbtion
    return new_x, new_P

# For plotting Gaussians: certainty of measurements (P-matrix)
def f(u, sigma2, x):
    return 1/np.sqrt(2*np.pi*sigma2) * np.exp(-0.5* ((x-u)**2/sigma2))
    
    

**Kalman filter model**, simple 2d model of world.
- x,y, vx, vy.
- Constant speed, ax, ay = 0


In [4]:
### Initialize Kalman filter ###
# The initial state (4x1).
x = np.array([[0],  # Position along the x-axis
              [0],  # Velocity along the x-axis 
              [0],  # Position along the y-axis
              [0]]) # Velocity along the y-axis 


# The initial uncertainty (4x4) - We start with some very large values.
# Then we will fully trust measurements first iteration.
P = np.array([[1000, 0, 0, 0, 0, 0],
              [0, 1000, 0, 0, 0, 0],
              [0, 0, 1000, 0, 0, 0],
              [0, 0, 0, 1000, 0, 0]])

# The external motion (4x1).
u = np.array([[0], [0], [0], [0]])  # - no external motion 

# The transition matrix (4x4). 
F = np.array([[1, 1, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 1],
              [0, 0, 0, 1]])

# The observation matrix (4x4).
H = np.identity(4)              #H = np.array([[1, 0, 0, 0],
                                #              [0, 1, 0, 0]])

# The measurement uncertainty.
# How little do you want to trust the measurements?
R = 1*np.array([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])

# Disturbance matrix
# How little do you want to trust the model?
Q = 1*np.array([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])

***TODO***
- Multiobject tracking: stop tracking when object leaves frame, and restart tracking (restart x, P matrices)

In [ ]:
running = True
x_range = np.arange(-10, 30, 0.1) # Range of possible xpos
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(16,6))

# Measurement and state history
zs = []
xs = []

# Flag for new measurements
new_measurement = None

while running:
    if new_measurement:
        # Fresh measurements!
        z = np.array([[...],   # xpos
                    [...],   # velx
                    [...],   # ypos
                    [...]])  # vely
        zs.append(z)

        # Update based on new measurements + previous prediction
        x, P = update(x, P, z, H, R)
    else:
        zs.append([None])


    # Predict based on model
    x, P = predict(x, P, F, u, Q)
    xs.append(x)


    # Do some plotting
    #plt.plot(axis, f(x[0], P[0, 0], axis)) # PosX
    ax[0, 0].plot(x_range, f(x[0], P[0, 0], x_range)) # PosX
    ax[0, 1].plot(x_range, f(x[1], P[1, 1], x_range)) # VelX
    ax[1, 0].plot(x_range, f(x[2], P[2, 2], x_range)) # PosY
    ax[1, 1].plot(x_range, f(x[3], P[3, 3], x_range)) # VelY

# plt.title("xpos")
# plt.show()
ax[0, 0].set_title("x pos")
ax[0, 1].set_title("x vel")
ax[1, 0].set_title("y pos")
ax[1, 1].set_title("y vel")